In [27]:
# DEVOIR 2
# PARTIE IV: VISUALISATION ET REPRÉSENTATIONS
import numpy as np
import pandas as pd
import plotly.express as px

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

In [28]:
data=pd.read_csv('dataB.csv', index_col=0)
x = data.values
countries = data.index.values
countries = pd.DataFrame(countries, columns=['COUNTRY'])
countries

,COUNTRY
0,Afghanistan
1,Albania
2,Algeria
3,Angola
4,Argentina
...,...
152,Venezuela
153,Vietnam
154,Yemen
155,Zambia


In [29]:
def pca_data(x, components):
    pca = PCA(n_components = components)
    pc = pca.fit_transform(x)
    
    print(pca.explained_variance_ratio_.cumsum())
    
    if components == 2:
        pc_df = pd.DataFrame(data = pc, columns=['PC1', 'PC2'])
    else:
        pc_df = pd.DataFrame(data = pc, columns=['PC1', 'PC2', 'PC3', 'PC4', 'PC5'])
    
    pc_df = pd.concat([countries, pc_df],axis = 1)
    return pc_df

In [30]:
def tSNE_data(x, components):
    tsne = TSNE(n_components=components, verbose=1, perplexity=40, n_iter=300)
    tsne_data = tsne.fit_transform(x)
    tsne_df = pd.DataFrame(data = tsne_data, columns=['Axis 1', 'Axis 2'])
    tsne_df = pd.concat([countries, tsne_df],axis = 1)
    return tsne_df

In [31]:
# 2D PCA Transformation


x = StandardScaler().fit_transform(x)
modified_data1 = pca_data(x, 2)

fig = px.scatter(modified_data1, x='PC1', y='PC2', text='COUNTRY')
fig.update_traces(textposition='top center')
fig.update_layout(title_text='2D PCA Transformation', title_x=0.5)
fig.update_layout(
    autosize=False,
    width=2000,
    height=1000
)
fig.show()

# We realized that the structure is not linear, more like geodesic.
# Let's try Isomap this time.

[0.38348611 0.46646909]


In [32]:
np.random.seed(101)
modified_data3 = tSNE_data(pca_data(x,5).iloc[:,1:].values, 2)

fig = px.scatter(modified_data3, x='Axis 1', y='Axis 2', text='COUNTRY')
fig.update_traces(textposition='top center')
fig.update_layout(title_text='2D t-SNE Transformation with PCA 5 dimensions', title_x=0.5)
fig.update_layout(
    autosize=False,
    width=2000,
    height=1000
)
fig.show()

[0.38348611 0.46646909 0.5343968  0.59352889 0.64321605]
[t-SNE] Computing 121 nearest neighbors...
[t-SNE] Indexed 157 samples in 0.001s...
[t-SNE] Computed neighbors for 157 samples in 0.004s...
[t-SNE] Computed conditional probabilities for sample 157 / 157
[t-SNE] Mean sigma: 2.647569
[t-SNE] KL divergence after 250 iterations with early exaggeration: 53.721710
[t-SNE] KL divergence after 300 iterations: 0.306002


In [ ]:
'''
Analyse sur la réduction de dimensionnalité par rapport aux données de dataB.csv

a) Nous nous sommes servis de la régression linéaire afin de faire de l'imputation, donc
nous pensons que les données peuvent avoir une "structure" linéaire. Par conséquent,
notre première hypothèse est d'utiliser la décomposition PCA en 2 et 5 dimensions.

[Image de PCA en 2 dimensions]

b) Pour PCA: on constate que PCA en 2 dimensions ne fournit pas assez d'informations par rapport
aux données, donc il se peut la "structure" de cette banque de données n'est pas linéaire. Ceci
a du sens étant donné que certaines valeurs générées par la régression linéaire nous donne des valeurs
absurdes. Par exemple, des valeurs négatives pour un attribut qui contient uniquement des valeurs positives
(valeur monétaire par exemple). La décomposition PCA de deux dimensions conserve uniquement 38.3%
de la variance, tandis que PCA de cinq dimensions conserve 64.3% de variance. En conclusion, même pour
PCA de 5 dimensions, nous n'avons toujours pas atteint une variance de 95% ou plus, donc PCA n'est pas à tout fait
la méthode idéale pour cette banque de données.

Nouvelle tentative: Utiliser t-SNE. 

La méthode t-SNE probabiliste est idéale lorsque les données ne forment pas une
"structure" linéaire. Elle permet aussi de visualiser de façon claire les données avec un grand nombre
d'attributs (features). Cependant, il faut appliquer une autre réduction de dimensionnalité sur t-SNE, puisque
cet algorithme a une très grande complexité du temps. Nous avons décidé de réduire la dimension de façon suivante:
1. Utiliser PCA de 5 dimensions sur les données initiales.
2. Appliquer t-SNE sur les données transformées par PCA.

[Image de t-SNE et PCA-5 en 2 dimensions]

Nous avons choisi PCA-5 puisque la variance est plus élevée que PCA-2, donc on obtient plus d'informations
après avoir extrait les données vers une plus petite dimension.

Dans le graphique, on peut voir déjà 4 groupes qui se forment, donc on peut donner quelques interprétations
par rapport à nos données. Lorsque deux pays se trouvent dans le même groupe, on peut suggérer l'idée
qu'ils ont des comportement assez similaires. Voici la liste des groupes:

Groupe 1: En bas à gauche: Pays d'Afrique
Groupe 2: À gauche (en haut du groupe 1): Pays d'Asie et quelques pays d'Afrique (Afrique du Sud, Algérie)
Groupe 3: Au centre: Amérique Centrale et du Sud à l'exception de Fiji, Seychelles, Botswana, Philippines, etc.
Groupe 4: En haut à droite: Amérique du Nord, Europe, Océanie et quelques pays d'Asie (Chine, Corée du Sud, Kazakhstan, etc.)
Le groupe 4 est assez grand. Par exemple la Chine est la Corée du Sud appartiennent dans le même groupe, mais les deux pays
ont des comportements qui semblent assez différents.

Même si regrouper les pays en 4 groupes peut être une généralisation assez exagérée, cette observation est beaucoup plus idéale
que la décomposition PCA en deux dimensions. Une proposition d'amélioration serait d'augmenter le nombre de dimensions de PCA
(par exemple, de 5 à 15 dimensions par exemple) avant d'utiliser t-SNE afin de maximiser la variance et d'avoir une 
interprétation plus légitime.

Source: https://towardsdatascience.com/visualising-high-dimensional-datasets-using-pca-and-t-sne-in-python-8ef87e7915b

'''